In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import warnings
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(color_codes=True)
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from scipy import stats
%matplotlib inline
warnings.filterwarnings("ignore")

## Importing all dataframes and Cleaning

### Income as "I"

In [2]:
#importing data frames, matching their postcode formats and making the postcode into index
income = pd.read_csv('data/Source 1  Australia (based on tax income bracket)/ts18individual25countaveragemedianbypostcode.csv',converters={'Postcode': '{:0>4}'.format})
income.index = income['Postcode']
income = income.drop(columns = ['Postcode'])

income.shape

(2470, 33)

### Population as "P"

In [3]:
#importing data set and skipping footer information which is not part of dataset
popul_s = pd.read_excel('data/Source 2 suburb info/Suburbs to Postcodes to Suburbs.xlsx', sheet_name="Postcode Population", index_col = 'Postcode', skipfooter = 16)

popul_s.shape

(2514, 1)

### Medicare as "M"

In [4]:
Medicare_NetTax = pd.read_excel('data/Source 5 - Medicare and Net Tax/Medicare and Net Tax.xlsx',converters={'Postcode': '{:0>4}'.format})
Medicare_NetTax.index = Medicare_NetTax['Postcode']
Medicare_NetTax = Medicare_NetTax.drop(columns = ['Postcode'])

#droping usless columns to reduce computational complexity during modeling. first lets remove state data because 
# other data sets that we use are only based on NSW and we are planning on using inner join for joining 
# datasets, thus the "state" attribute will only be NSW which is usless information
Medicare_NetTax = Medicare_NetTax.drop(columns = ['State/ Territory1'])

Medicare_NetTax.head()

,Number of individuals\nno.,Taxable income or loss3\nno.,Taxable income or loss3\n$,Tax on taxable income\nno.,Tax on taxable income\n$,Medicare levy\nno.,Medicare levy\n$,Medicare levy surcharge\nno.,Medicare levy surcharge\n$,Total Medicare levy liability\nno.,Total Medicare levy liability\n$,Net tax\nno.,Net tax\n$,Australian government allowances and payments\nno.,Australian government allowances and payments\n$
Postcode,,,,,,,,,,,,,,,
1225,54,54,8934191,45,3165101.59,39,167269.92,2,49.10,39,167319.02,44,3253650.53,0,0
1230,72,70,5308341,60,1402287.38,50,101041.27,0,0.00,50,101041.27,54,1466487.91,2,19797
1235,337,336,13019485,226,2779146.79,80,160680.85,4,6118.18,80,166799.03,192,2864291.49,6,40676
1240,68,65,4771650,52,1368038.93,38,82541.50,1,1317.37,38,83329.47,49,1411662.29,1,2916
1335,83,82,6542194,70,1715992.27,59,118836.86,3,4502.15,59,123339.01,65,1822576.18,2,10439


### House-Sales as "S"

In [5]:
sales = pd.read_excel('data/Source 4 - rent and sale by postcode/sales_info.xlsx',sheet_name="Postcode", header = 6,converters={'Postcode': '{:0>4}'.format})

sales.index = sales['Postcode']
sales = sales.drop(columns = ['Postcode'])

In [6]:
#remove columns that are not needed because they take into account quarterly metrics while only annual once
#are only useful to the annalysis as the data in this table should be a high level summary.
sales = sales.drop(columns = ["First Quartile Sales Price\n$'000s"])
sales = sales.drop(columns = ["Third Quartile Sales Price\n'000s"])
sales = sales.drop(columns = ["Qtly change in Count"])
sales = sales.drop(columns = ["Qtly change in Median"])
sales.shape

(1361, 6)

In [7]:
#now lets remove the colomn "dwelling" because it seems that this additional detail about sales for each property 
#type is not very relevant as we have the total thus only total aggregations will be considered.
sales = sales[(sales["Dwelling"] == "Total")]
sales = sales.drop(columns = ["Dwelling"])
sales.shape

(520, 5)

In [8]:
#replacing all missing values that represent count by approximate amount they are likely to be. 
#This info was derived from datasource info
sales["Sales\nNo."] = sales["Sales\nNo."].replace({"-": 5, "s": 20})
sales.shape

(520, 5)

In [9]:
# replacing emplty values for changes with 0 to ensure neutrality of null values and thus better accuracy
sales['Annual change in Median'] = sales["Annual change in Median"].replace({"-": 0, "s": 0})
sales['Annual change in Count'] = sales['Annual change in Count'].replace({"-": 0, "s": 0})
sales.shape

(520, 5)

In [10]:
# replacing the median statistic with its statistical mean
#to do that, first all null values have to be reaplced with some number to allow for future mean calculaion
#first replacing all "-" and "s" labels with NaN values
sales["Median Sales Price\n$'000s"] = sales["Median Sales Price\n$'000s"].replace({"-": np.NaN, "s": np.NaN})
# now calculating the mean of the column while exclusing NaN values
mean = sales["Median Sales Price\n$'000s"].mean(skipna = True)
# replacing all NaN values with mean which is a reliable NaN replacement
sales["Median Sales Price\n$'000s"] = sales["Median Sales Price\n$'000s"].replace({np.NaN: round(mean)})
sales.shape

(520, 5)

In [11]:
#now lets repeat the above procedure for the mean sales price
sales["Mean Sales Price\n$'000s"] = sales["Mean Sales Price\n$'000s"].replace({"-": np.NaN, "s": np.NaN})
# now calculating the mean of the column while exclusing NaN values
mean = sales["Mean Sales Price\n$'000s"].mean(skipna = True)
# replacing all NaN values with mean which is a reliable NaN replacement
sales["Mean Sales Price\n$'000s"] = sales["Mean Sales Price\n$'000s"].replace({np.NaN: round(mean)})
sales.shape

(520, 5)

In [12]:
# now grouping all postcodes together using different aggregation types for different attributes
sales_1 = sales[["Median Sales Price\n$'000s", "Mean Sales Price\n$'000s","Annual change in Median","Annual change in Count"]]
sales_1 = sales_1.groupby(by = sales.index).mean()

sales_2 = sales[["Sales\nNo."]]
sales_2 = sales_2.groupby(by = sales_2.index).sum()
sales.shape

(520, 5)

In [13]:
#joining all those cleaned sub tables together into a single table
sales = pd.merge(left= sales_1, right= sales_2, how='inner', left_index=True, right_index=True)

#changing names of columns
sales = sales.rename(columns={"Annual change in Median": "Annual median salesprice change", "Annual change in Count" : "Annual change in sales Count"})

sales.shape

(520, 5)

In [14]:
sales.head()

,Median Sales Price\n$'000s,Mean Sales Price\n$'000s,Annual median salesprice change,Annual change in sales Count,Sales\nNo.
Postcode,,,,,
2000,1225.0,1541.0,0.0524,-0.0278,105
2007,745.0,834.0,0.1622,0.1765,20
2008,750.0,956.0,-0.1269,0.6667,35
2009,986.0,1277.0,-0.1034,0.5789,20
2010,1280.0,1357.0,0.4222,0.9114,151


### House-Rent as "R" (STATUS: cleaned, no categorical)

In [15]:
rent = pd.read_excel('data/Source 4 - rent and sale by postcode/Issue-132-2020-Rent-tables-June-Quarter-2020.xlsx',sheet_name="Postcode",header = 5,converters={'Postcode': '{:0>4}'.format})

rent.index = rent['Postcode']
rent = rent.drop(columns = ['Postcode'])

In [16]:
#replacing all missing values that represent count by approximate amount they are likely to be. 
#This info was derived from datasource info
rent["New Bonds Lodged\nNo."] = rent["New Bonds Lodged\nNo."].replace({"-": 5, "s": 20})
rent["Total Bonds Held\nNo."] = rent["Total Bonds Held\nNo."].replace({"-": 5, "s": 20})
rent.shape

(16335, 11)

In [17]:
# replacing emplty values for changes with 0 to ensure neutrality of null values and thus better accuracy
rent['Annual change in Median Weekly Rent'] = rent["Annual change in Median Weekly Rent"].replace({"-": 0, "s": 0})
rent['Annual change in New Bonds Lodged'] = rent['Annual change in New Bonds Lodged'].replace({"-": 0, "s": 0})
rent.shape

(16335, 11)

In [18]:
# replacing the median statistic with its statistical mean
#to do that, first all null values have to be reaplced with some number to allow for future mean calculaion
#first replacing all "-" and "s" labels with NaN values
rent['Median Weekly Rent for New Bonds\n$'] = rent['Median Weekly Rent for New Bonds\n$'].replace({"-": np.NaN, "s": np.NaN})
# now calculating the mean of the column while exclusing NaN values
mean = rent['Median Weekly Rent for New Bonds\n$'].mean(skipna = True)
# replacing all NaN values with mean which is a reliable NaN replacement
rent['Median Weekly Rent for New Bonds\n$'] = rent['Median Weekly Rent for New Bonds\n$'].replace({np.NaN: round(mean)})
rent.shape

(16335, 11)

In [19]:
#usless column. Annual metrics are superior
rent = rent.drop(columns = ['Quarterly change in New Bonds Lodged'])
rent = rent.drop(columns = ['Quarterly change in Median Weekly Rent'])
rent.shape

(16335, 9)

In [20]:
# these columns are not necessary for analysis becuase the median number is given thus will be deleted.
rent = rent.drop(columns = ['First Quartile Weekly Rent for New Bonds\n$'])
rent = rent.drop(columns = ['Third Quartile Weekly Rent for New Bonds\n$'])
rent.shape

(16335, 7)

In [21]:
# removing this data because in my opinion the number of rooms in the house will have more impact on the quality of 
#analysis when compared to the acocmodation type which "dwelling types " column indicates. The problem with 
#incorporating both of those is because together they are extremely hard to integrate together thus this one 
#is deleted
rent = rent[(rent["Dwelling Types"] == "Total")]
rent = rent.drop(columns = ["Dwelling Types"])
rent.shape

(3892, 6)

In [22]:
#preparing this data frame for integration with other data frames by extracting the column information into rows 
# and grouping all postcodes together
rent_0 = rent[["Number of Bedrooms", "Total Bonds Held\nNo."]]
rent_0 =pd.pivot_table(rent_0, index = rent_0.index,columns='Number of Bedrooms',values='Total Bonds Held\nNo.',aggfunc= sum,fill_value=0)
rent.shape

(3892, 6)

In [23]:
# further refining rent data frame to integrate it with rent_second dataframe
rent = rent.drop(columns = ["Number of Bedrooms"])
rent.shape

(3892, 5)

In [24]:
# now grouping all postcodes together using different aggregation types for different attributes
rent_1 = rent[["Median Weekly Rent for New Bonds\n$", "Annual change in Median Weekly Rent", "Annual change in New Bonds Lodged"]]
rent_1 = rent_1.groupby(by = rent.index).mean()

rent_2 = rent[["New Bonds Lodged\nNo.", "Total Bonds Held\nNo."]]
rent_2 = rent_2.groupby(by = rent_2.index).sum()
rent.shape

(3892, 5)

In [25]:
#joining all those cleaned sub tables together into a single table
rent = pd.merge(left= rent_0, right= rent_1, how='left', left_index=True, right_index=True)
rent = pd.merge(left= rent, right= rent_2, how='left', left_index=True, right_index=True)

#renaming colomn names because sales table has same names for number of bedroom colomns and they need to be
#distinguishable when dataframes are merged
rent = rent.rename(columns={"1 Bedroom": "No. 1 Bedroom rented", "2 Bedrooms": "No. 2 Bedroom rented","3 Bedrooms": "No. 3 Bedroom rented", "4 or more Bedrooms": "No. 4 or more Bedrooms rented", "Bedsitter": "No. Bedsitter rented","Not Specified": "No. Not Specified rented" , "Total": "No. Total rents"})

rent.shape

(617, 12)

In [26]:
rent.head()

,No. 1 Bedroom rented,No. 2 Bedroom rented,No. 3 Bedroom rented,No. 4 or more Bedrooms rented,No. Bedsitter rented,No. Not Specified rented,No. Total rents,Median Weekly Rent for New Bonds\n$,Annual change in Median Weekly Rent,Annual change in New Bonds Lodged,New Bonds Lodged\nNo.,Total Bonds Held\nNo.
Postcode,,,,,,,,,,,,
2000,3445,2930,416,42,232,530,7595,711.428571,-0.177900,-0.017729,1866,15190
2007,961,607,119,20,20,74,1810,564.857143,-0.009300,0.238571,458,3611
2008,2106,791,233,117,605,434,4286,639.000000,-0.094186,-0.091743,941,8572
2009,886,1392,326,20,38,134,2793,576.285714,-0.153414,1.504343,857,5589
2010,3535,2267,625,286,609,757,8079,641.142857,-0.107971,0.290086,1898,16158


### Speeding Tickets as "Sp"

In [27]:
#importing data frame
ColumnNameList = ["Postcode", "Speed Camera Fines 2015", "Police Fines 2015"]
speedingtickets = pd.read_excel('data/source 3 - speeding info/NSW postcod, Speeding.xlsx', names = ColumnNameList,usecols = "B,F,J", header = 17,converters={'Postcode': '{:0>4}'.format})

#setting index and creating a summary column
speedingtickets.index = speedingtickets['Postcode']
speedingtickets["total speeding fines 2015"] = speedingtickets["Speed Camera Fines 2015"] + speedingtickets["Police Fines 2015"]

#deleting all unnecessary data
speedingtickets = speedingtickets.drop(columns = ['Postcode'])
speedingtickets = speedingtickets.drop(columns = ['Speed Camera Fines 2015'])
speedingtickets = speedingtickets.drop(columns = ['total speeding fines 2015'])

#renaming colomns 
speedingtickets = speedingtickets.rename(columns={"Police Fines 2015": "sum of speeding fines paid in 2015($)"})

speedingtickets.shape

(596, 1)

In [28]:
speedingtickets.head()

,sum of speeding fines paid in 2015($)
Postcode,
2170,1157473
2145,582823
2560,862292
2148,636434
2200,470023


## Merging all dataframes

In [29]:
# everything is joint using inner joint because there is abundance of information avaialble and thus instead of 
# having to fill missing values using KNN, we can just drop those rows that dont have an index in both dataframes
#we do this because this reduces the complexity of the task and doesnt significantly compromise on later results 
# because there are still many rows avaialble to use for analysis.
I_P = pd.merge(left= income, right= popul_s, how='inner', left_index=True, right_index=True)
I_P.shape

(2470, 34)

In [30]:
I_P.head()

,Number of individuals lodging an income tax return,Count taxable income or loss,Average taxable income or loss,Median taxable income or loss,Proportion with salary or wages,Count salary or wages,Average salary or wages,Median salary or wages,Proportion with net rent,Count net rent,...,Average total business expenses,Median total business expenses,Proportion with net tax,Count net tax,Average net tax,Median net tax,Count super total accounts balance,Average super total accounts balance,Median super total accounts balance,Population
Postcode,,,,,,,,,,,,,,,,,,,,,
0800,5168,5168,74990,56363,0.92,4777,76428,59845,0.14,741,...,55931,10500,0.87,4475,22583,12806,7324,74077,11624,4545
0810,21449,21449,66197,55311,0.87,18675,67931,59999,0.20,4228,...,58678,12751,0.82,17484,18948,13302,25039,119697,42545,29723
0812,11170,11170,64626,56496,0.88,9797,67487,60989,0.20,2266,...,56098,16228,0.83,9226,17661,13313,13262,126783,53552,19315
0820,13186,13186,80947,63161,0.89,11690,77029,65904,0.23,3047,...,60852,13101,0.85,11271,25007,15371,15877,143697,46748,19049
0822,6279,6279,48181,36292,0.91,5707,46538,35992,0.10,642,...,55751,16282,0.68,4256,13117,7785,9351,49967,10471,24111


In [31]:
I_P_M = pd.merge(left= I_P, right= Medicare_NetTax, how='inner', left_index=True, right_index=True)
I_P_M.shape

(602, 49)

In [32]:
I_P_M.head()

,Number of individuals lodging an income tax return,Count taxable income or loss,Average taxable income or loss,Median taxable income or loss,Proportion with salary or wages,Count salary or wages,Average salary or wages,Median salary or wages,Proportion with net rent,Count net rent,...,Medicare levy\nno.,Medicare levy\n$,Medicare levy surcharge\nno.,Medicare levy surcharge\n$,Total Medicare levy liability\nno.,Total Medicare levy liability\n$,Net tax\nno.,Net tax\n$,Australian government allowances and payments\nno.,Australian government allowances and payments\n$
Postcode,,,,,,,,,,,,,,,,,,,,,
2000,23478,23478,73334,29450,0.82,19149,58276,30795,0.13,3032,...,9629,24540132.04,359,707366.22,9624,25225846.88,16256,4.831947e+08,279,1743683
2006,74,74,65946,36098,0.81,60,70665,43594,0.23,17,...,41,99085.96,0,0.00,41,99085.96,50,1.644460e+06,3,28769
2007,5032,5032,47170,29967,0.85,4301,49272,33707,0.13,646,...,2416,3637206.66,73,81471.59,2411,3713341.81,3412,5.294173e+07,215,1191725
2008,6305,6305,58973,41681,0.87,5483,57918,45454,0.12,773,...,3633,6984279.64,154,185902.68,3640,7170182.32,4549,1.125195e+08,319,1802427
2009,9688,9688,86162,53780,0.82,7930,79347,58978,0.18,1762,...,6218,13916952.11,227,401735.62,6209,14253050.03,7691,2.334525e+08,213,1224571


In [33]:
I_P_M_S = pd.merge(left= I_P_M, right= sales, how='inner', left_index=True, right_index=True)
I_P_M_S.shape

(520, 54)

In [34]:
I_P_M_S.head()

,Number of individuals lodging an income tax return,Count taxable income or loss,Average taxable income or loss,Median taxable income or loss,Proportion with salary or wages,Count salary or wages,Average salary or wages,Median salary or wages,Proportion with net rent,Count net rent,...,Total Medicare levy liability\n$,Net tax\nno.,Net tax\n$,Australian government allowances and payments\nno.,Australian government allowances and payments\n$,Median Sales Price\n$'000s,Mean Sales Price\n$'000s,Annual median salesprice change,Annual change in sales Count,Sales\nNo.
Postcode,,,,,,,,,,,,,,,,,,,,,
2000,23478,23478,73334,29450,0.82,19149,58276,30795,0.13,3032,...,25225846.88,16256,4.831947e+08,279,1743683,1225.0,1541.0,0.0524,-0.0278,105
2007,5032,5032,47170,29967,0.85,4301,49272,33707,0.13,646,...,3713341.81,3412,5.294173e+07,215,1191725,745.0,834.0,0.1622,0.1765,20
2008,6305,6305,58973,41681,0.87,5483,57918,45454,0.12,773,...,7170182.32,4549,1.125195e+08,319,1802427,750.0,956.0,-0.1269,0.6667,35
2009,9688,9688,86162,53780,0.82,7930,79347,58978,0.18,1762,...,14253050.03,7691,2.334525e+08,213,1224571,986.0,1277.0,-0.1034,0.5789,20
2010,20764,20764,89782,59960,0.86,17766,83715,63647,0.15,3114,...,33595820.00,17198,5.390736e+08,765,4708267,1280.0,1357.0,0.4222,0.9114,151


In [35]:
I_P_M_S_R = pd.merge(left= I_P_M_S, right= rent, how='inner', left_index=True, right_index=True)
I_P_M_S_R.shape

(520, 66)

In [36]:
I_P_M_S_R.head()

,Number of individuals lodging an income tax return,Count taxable income or loss,Average taxable income or loss,Median taxable income or loss,Proportion with salary or wages,Count salary or wages,Average salary or wages,Median salary or wages,Proportion with net rent,Count net rent,...,No. 3 Bedroom rented,No. 4 or more Bedrooms rented,No. Bedsitter rented,No. Not Specified rented,No. Total rents,Median Weekly Rent for New Bonds\n$,Annual change in Median Weekly Rent,Annual change in New Bonds Lodged,New Bonds Lodged\nNo.,Total Bonds Held\nNo.
Postcode,,,,,,,,,,,,,,,,,,,,,
2000,23478,23478,73334,29450,0.82,19149,58276,30795,0.13,3032,...,416,42,232,530,7595,711.428571,-0.177900,-0.017729,1866,15190
2007,5032,5032,47170,29967,0.85,4301,49272,33707,0.13,646,...,119,20,20,74,1810,564.857143,-0.009300,0.238571,458,3611
2008,6305,6305,58973,41681,0.87,5483,57918,45454,0.12,773,...,233,117,605,434,4286,639.000000,-0.094186,-0.091743,941,8572
2009,9688,9688,86162,53780,0.82,7930,79347,58978,0.18,1762,...,326,20,38,134,2793,576.285714,-0.153414,1.504343,857,5589
2010,20764,20764,89782,59960,0.86,17766,83715,63647,0.15,3114,...,625,286,609,757,8079,641.142857,-0.107971,0.290086,1898,16158


In [37]:
I_P_M_S_R_Sp = pd.merge(left= I_P_M_S_R, right= speedingtickets, how='inner', left_index=True, right_index=True)
I_P_M_S_R_Sp.shape

(512, 67)

In [38]:
# displaying the combined data and giving the combined data frame a simpler name
I_P_M_S_R_Sp.head()
df = I_P_M_S_R_Sp

## Further Data Preparation

- Now, when the data has been joint and all primitive cleaning has been done, we can start to do more advanced data cleaning and preparation

In [39]:
# because of causation issues discussed during the exploration phase which occured after
# the data preparation and cleaning, following colomns are being removed.
df = df.drop(["Median taxable income or loss"], axis =1)
df = df.drop(["Average taxable income or loss"], axis =1)
df = df.drop(["Median total income or loss"], axis =1)
    
df = df.drop(["Average salary or wages"], axis =1)
df = df.drop(["Median salary or wages"], axis =1)

df = df.drop(['Average net tax'], axis =1)
df = df.drop(['Median net tax'], axis =1)

df = df.drop(['Average super total accounts balance'], axis =1)
df = df.drop(['Median super total accounts balance'], axis =1)

#list of colomns to be used for modeling
df.columns

Index(['Number of individuals lodging an income tax return',
       'Count taxable income or loss', 'Proportion with salary or wages',
       'Count salary or wages', 'Proportion with net rent', 'Count net rent',
       'Average net rent', 'Median net rent', 'Count total income or loss',
       'Average total income or loss', 'Count total deductions',
       'Average total deductions', 'Median total deductions',
       'Proportion with total business income', 'Count total business income',
       'Average total business income', 'Median total business income',
       'Proportion with total business expenses',
       'Count total business expenses', 'Average total business expenses',
       'Median total business expenses', 'Proportion with net tax',
       'Count net tax', 'Count super total accounts balance', 'Population',
       'Number of individuals\nno.', 'Taxable income or loss3\nno.',
       'Taxable income or loss3\n$', 'Tax on taxable income\nno.',
       'Tax on taxable incom

In [40]:
df.tail()

,Number of individuals lodging an income tax return,Count taxable income or loss,Proportion with salary or wages,Count salary or wages,Proportion with net rent,Count net rent,Average net rent,Median net rent,Count total income or loss,Average total income or loss,...,No. 4 or more Bedrooms rented,No. Bedsitter rented,No. Not Specified rented,No. Total rents,Median Weekly Rent for New Bonds\n$,Annual change in Median Weekly Rent,Annual change in New Bonds Lodged,New Bonds Lodged\nNo.,Total Bonds Held\nNo.,sum of speeding fines paid in 2015($)
Postcode,,,,,,,,,,,,,,,,,,,,,
2869,714,714,0.64,455,0.11,80,3846,2228,714,42406,...,5,5,5,60,502.000000,0.000000,0.000000,35,132,16548
2870,6334,6334,0.79,5014,0.13,818,601,-456,6334,53830,...,178,5,101,974,389.142857,0.008100,0.020129,202,1943,130811
2871,4851,4851,0.74,3596,0.13,623,2294,797,4851,53060,...,93,5,20,714,374.571429,0.056414,0.068943,188,1436,116948
2877,1937,1937,0.71,1371,0.09,173,2139,762,1937,48284,...,20,0,20,199,459.166667,0.003467,0.092600,45,402,43406
2880,8815,8815,0.84,7430,0.13,1111,-831,-337,8815,57767,...,159,0,49,1621,313.166667,-0.023017,-0.086350,252,3242,106930


## Adopting data for categorical analysis

Because all data is numerical in the current data set and one of the models that we are trying to produce requires classification. Thus we need to convert our numberical y variable into classes. Lets create 2 classes such as;
- non-high_income: x < 80k
- high_income: 80k =< x

Those classes are assigned to suburbs with according average income levels and we will be creating a model that outputs a probability that an individual living in provided suburb has _high_income_ label

Note: _x_ represents average income or loss

Now,lets impliment those changes into our dataframe

In [41]:
df["income status"] = df["Average total income or loss"]
def assign(x):
    if x < 80000:
        return "non-high_income"
    if 80000 <= x:
        return "high_income"

#lambda is an anonymous function that applies the assign function to all x values in "income status" column 
df["income status"] = df["income status"].apply(lambda x: assign(x))

In [42]:
df.groupby("income status").size()

income status
high_income         76
non-high_income    436
dtype: int64

As can be seen above, we have a heathy distribution of data as the high_income class has enough data points to ensure that machine learning performs reasonably well. It would be a bad idea to brake down income status into more than 2 classes because than each individual class would get too small for prediction to be accurate, thus we need to keep our labels very general. In addition, it is projected that the model will be better at predicting non-high income postcode because there are a lot more non-high income postcodes labels.

## Splitting into x and y

In [43]:
# separating data into 3 groups which are features X and 2 types of things to 
# be predicted (categorical and numeric income)

X = df.drop(["income status","Average total income or loss"], axis = 1)
y_cat = df["income status"]
y_reg = df["Average total income or loss"]

## Standartising the numeric data

Doing standartisation on the feature data because one of the algortihms being used is KNN and it depends on the distance metric which is highly impacted by incosistent scale of data and thus standartisation was used to scale all data and make it more comparable. Also standardtisation was used because RFE is used to optimise hyperparameters for Logistic regression and/or Linear Regression and the accuract of RFE is highly impacted by data that has inconsistent scale. Lastly, Normalisation was also attempted on the data, yet performance metrics were not as good as performance metrics produced by standardtisation model.

In [44]:
# saving column names because standartisation procedure will remove them as it turns the dataframe into np array
saved_cols_X = X.columns.tolist()
saved_cols_y_reg = ["Average total income or loss"]

# standartising the data
X = stats.zscore(X)
y_reg = stats.zscore(y_reg)

## Saving the file as csv

In [45]:
#converting the standardised array back to dataframe and reassigning column names
X = pd.DataFrame(X)
X.columns = saved_cols_X
#re-specfying the index because initial index was removed due to standartisation
X.index = df.index
X.to_csv('df_X.csv')
X

,Number of individuals lodging an income tax return,Count taxable income or loss,Proportion with salary or wages,Count salary or wages,Proportion with net rent,Count net rent,Average net rent,Median net rent,Count total income or loss,Count total deductions,...,No. 4 or more Bedrooms rented,No. Bedsitter rented,No. Not Specified rented,No. Total rents,Median Weekly Rent for New Bonds\n$,Annual change in Median Weekly Rent,Annual change in New Bonds Lodged,New Bonds Lodged\nNo.,Total Bonds Held\nNo.,sum of speeding fines paid in 2015($)
Postcode,,,,,,,,,,,,,,,,,,,,,
2000,1.821440,1.821440,0.761707,1.834628,-0.699645,1.145399,-0.617507,-0.796766,1.821440,1.821440,...,-0.568031,2.971010,3.165302,3.433943,2.529313,-5.093835,-0.252826,4.696054,3.434369,0.395283
2007,-0.442704,-0.442704,1.173963,-0.377646,-0.699645,-0.558050,-0.449235,-0.581142,-0.442704,-0.442704,...,-0.644711,0.019848,-0.147735,0.165977,0.773048,-0.069184,1.061550,0.508731,0.163017,-0.706619
2008,-0.286450,-0.286450,1.448801,-0.201535,-0.942835,-0.467381,-0.617878,-0.372173,-0.286450,-0.286450,...,-0.306623,8.163386,2.467821,1.564677,1.661451,-2.598965,-0.632391,1.945150,1.564622,-0.580753
2009,0.128794,0.128794,0.761707,0.163056,0.516303,0.238701,-0.826550,-0.634382,0.128794,0.128794,...,-0.644711,0.270418,0.288191,0.721277,0.909989,-4.364107,7.552767,1.695338,0.721851,-0.482224
2010,1.488311,1.488311,1.311382,1.628568,-0.213266,1.203941,-0.622696,-0.629724,1.488311,1.488311,...,0.282416,8.219069,4.814556,3.707356,1.687127,-3.009810,1.325729,4.791221,3.707853,-0.067633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2869,-0.972714,-0.972714,-1.711829,-0.950680,-1.186024,-0.962138,1.201611,1.638988,-0.972714,-0.972714,...,-0.696992,-0.188961,-0.649050,-0.822604,0.019875,0.207976,-0.161909,-0.749250,-0.819886,-0.765884
2870,-0.282891,-0.282891,0.349451,-0.271413,-0.699645,-0.435254,-0.001127,-0.147232,-0.282891,-0.282891,...,-0.094011,-0.188961,0.048431,-0.306282,-1.332414,0.449374,-0.058684,-0.252600,-0.308234,0.065721
2871,-0.464921,-0.464921,-0.337642,-0.482688,-0.699645,-0.574471,0.626372,0.686648,-0.464921,-0.464921,...,-0.390274,-0.188961,-0.540069,-0.453157,-1.507014,1.889246,0.191649,-0.294235,-0.451474,-0.035173


In [46]:
#converting the standardised array back to dataframe and reassigning column names
y_reg = pd.DataFrame(y_reg)
y_reg.columns = saved_cols_y_reg
y_reg.index = df.index
y_reg.to_csv('df_y_reg.csv')
y_reg

,Average total income or loss
Postcode,
2000,0.454341
2007,-0.613679
2008,0.276622
2009,0.963384
2010,1.101364
...,...
2869,-0.867348
2870,-0.424446
2871,-0.454298


In [47]:
#categorical variable doesnt need to be normalised
y_cat.to_csv('df_y_cat.csv')

## Evaluation of problems with source data and solutions

The original state of data was unsuable because of:
- inconsistent postcode formats which were reformatted by adding "0" padding (converting from varchar to char)
- some numeric values were replaced with "s" and "-" in the original dataset to prevent individual tracing. This was adressed by assigning those values a number based on case.
- data was stored in formatted excel documents with footers, various descriptions, headers and links. Those were adressed by defining dataframe import settings.
- data initially had a plethora of colomns that said very similar thing in different words. Those colomns were removed.
- datasets had varying numbers of postcodes and thus inner joint was used to avoid null values.
- some of the information provided was overly specific and thus was removed to avoid data overfitting
- Certain dataframes had multiple entries for each postcode which needed to be fixed though usse of pivot tables and selection of features.
- datasets were unconnected and thus unusuble which was adressed by joining them
- column headings lacked description and their meaning was ambiguous thus renaming on colomns was done.
- values within the dataframe had varying scales and thus standartisation was utilised to adress that.

As A result of data cleaning and preparation, 3 files were created such as:
- df_X : this file consists of normalised x feature values
- df_y_cat : this file consists of y categorical labels that were derived from y numerical values
- df_y_reg : this file consists of y numerical values